In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2024-10-10 14:10:16.453377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-10 14:10:16.681812: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-10 14:10:16.741741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 14:10:17.143804: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1728569425.563121      64 cuda_executor.c

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import VGG16
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
(x_train, y_train), (features_test, label_test)  = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [9]:
x_test, x_val, y_test, y_val = train_test_split(features_test, label_test, test_size=0.5, random_state=2)
x_test.shape, x_val.shape, y_test.shape, y_val.shape

((5000, 32, 32, 3), (5000, 32, 32, 3), (5000, 1), (5000, 1))

In [10]:
# Creating an Image data generator

train_img_gen = ImageDataGenerator(rescale=1./255
                                  , rotation_range=40
                                  ,width_shift_range=0.2
                                  ,height_shift_range=0.2
                                  ,shear_range=0.1
                                  ,zoom_range=0.1
                                  ,horizontal_flip=False
                                  ,fill_mode='nearest')

val_img_gen = ImageDataGenerator(rescale=1./255)

test_img_gen = ImageDataGenerator(rescale=1./255)

In [11]:
batch_size = 32

train_data_gen = train_img_gen.flow(x_train, y_train,batch_size=batch_size)
val_img_gen = val_img_gen.flow(x_val, y_val, batch_size=batch_size)
test_img_gen = test_img_gen.flow(x_test, y_test, batch_size=batch_size)

In [12]:
model = tf.keras.Sequential([
          layers.Conv2D(64, 3, activation='relu', input_shape=(32, 32 ,3))
        , layers.MaxPooling2D()
        , layers.Conv2D(128, 3, activation='relu')
        , layers.MaxPooling2D()
        , layers.Flatten()
        , layers.Dense(128, activation='relu')
        , layers.Dense(10, activation='softmax')])


optimizer = tf.keras.optimizers.Adam(0.001)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 666,890 (2.54 MB)

 Trainable params: 666,890 (2.54 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.fit(train_data_gen
          , steps_per_epoch=len(x_train)// batch_size
          , epochs=15
          , validation_data=val_img_gen
          , validation_steps=len(x_val)//batch_size)

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1728502478.684834      85 service.cc:146] XLA service 0x7ff0a8005010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728502478.685185      85 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2024-10-09 19:34:38.753010: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-09 19:34:38.974661: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906


  14/1562 ━━━━━━━━━━━━━━━━━━━━ 20s 13ms/step - accuracy: 0.0886 - loss: 2.3307  

I0000 00:00:1728502482.085076      85 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 28s 15ms/step - accuracy: 0.3057 - loss: 1.8718 - val_accuracy: 0.4952 - val_loss: 1.4365
Epoch 2/15
   1/1562 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.4688 - loss: 1.7609

...

1562/1562 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - accuracy: 0.5488 - loss: 1.2628 - val_accuracy: 0.6392 - val_loss: 1.0139
Epoch 12/15
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 11us/step - accuracy: 0.6250 - loss: 1.2318 - val_accuracy: 0.6250 - val_loss: 0.8425
Epoch 13/15
   5/1562 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.5249 - loss: 1.4000  

2024-10-09 19:37:03.226810: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 10785526222562568693
2024-10-09 19:37:03.226882: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 9505453121807118680
2024-10-09 19:37:03.236964: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 10785526222562568693
2024-10-09 19:37:03.237052: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 9505453121807118680


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - accuracy: 0.5662 - loss: 1.2119 - val_accuracy: 0.6434 - val_loss: 0.9891
Epoch 14/15
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 0s 9us/step - accuracy: 0.5000 - loss: 1.3532 - val_accuracy: 1.0000 - val_loss: 0.5044
Epoch 15/15
   5/1562 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - accuracy: 0.5601 - loss: 1.2762  

2024-10-09 19:37:27.133768: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 10785526222562568693
2024-10-09 19:37:27.133828: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 9505453121807118680
2024-10-09 19:37:27.140586: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 10785526222562568693
2024-10-09 19:37:27.140647: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 9505453121807118680


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 24s 15ms/step - accuracy: 0.5802 - loss: 1.1897 - val_accuracy: 0.6308 - val_loss: 1.0619


In [15]:

base_model = VGG16(input_shape=(32, 32, 3), include_top=False, weights='imagenet')

# Fine-tune from this layer onwards
unfreeze = 5

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:-unfreeze]:
  layer.trainable = False

base_model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 32, 32, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 8, 8, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 2, 2, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 7,079,424 (27.01 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

In [16]:
model2 = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(500, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [17]:
optimizer = tf.keras.optimizers.Adam(0.001)
model2.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model2.fit(train_data_gen
           , epochs=10
           , validation_data=val_img_gen
           , verbose=1
           , batch_size=128)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.0976 - loss: 2.3027 - val_accuracy: 0.0988 - val_loss: 2.3025
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.0971 - loss: 2.3028 - val_accuracy: 0.1018 - val_loss: 2.3023
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.0988 - loss: 2.3027 - val_accuracy: 0.1052 - val_loss: 2.3027
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.0975 - loss: 2.3028 - val_accuracy: 0.1052 - val_loss: 2.3026
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.0983 - loss: 2.3028 - val_accuracy: 0.0978 - val_loss: 2.3027
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.1010 - loss: 2.3027 - val_accuracy: 0.0974 - val_loss: 2.3027
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 26s 17ms/step - accuracy: 0.0958 - loss: 2.3028 - val_accuracy: 0.0978 - val_loss: 2.3026
Epoch 8/10
 229/1563 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.1089 -

In [19]:
tf.__version__

'2.17.0'